<a href="https://colab.research.google.com/github/Zewtta/PEL219_2025_RedesNeuraisArtificiais/blob/main/PEL219_Perceptron_Matheus_Vieira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Tarefa 1 - Perceptron**
##Aluno: Matheus Vieira

## Atividade 1 - Treinamento com o modelo .csv reduzido

### Teste perceptron com base no código de referência

In [48]:
import numpy as np
import pandas as pd

class Perceptron:
    def __init__(self, w=None):
        """
        w: vetor de pesos incluindo bias (w0, w1, w2, ...)
        Se None, será definido na primeira predição de acordo com #features.
        """
        self.w = None if w is None else np.asarray(w, dtype=float)

    @staticmethod
    def _add_bias(X: np.ndarray) -> np.ndarray:
        return np.concatenate([np.ones((X.shape[0], 1)), X], axis=1)

    @staticmethod
    def _heaviside(z: np.ndarray) -> np.ndarray:
        return (z >= 0).astype(int)

    def net(self, X: np.ndarray) -> np.ndarray:
        """
        Calcula a soma ponderada (net) para um conjunto de amostras X (sem bias).
        """
        Xb = self._add_bias(X)
        # se os pesos não existem ainda, inicializa com zeros compatíveis
        if self.w is None:
            self.w = np.zeros(Xb.shape[1], dtype=float)
        return Xb @ self.w

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Retorna a saída binária (0/1) aplicando a função degrau sobre net.
        """
        return self._heaviside(self.net(X))

# Lê CSV do GitHub
url = "https://raw.githubusercontent.com/Zewtta/PEL219_2025_RedesNeuraisArtificiais/e5a63434c13a8ac9a80d4d303b97a8c8de5b01d0/simple_diabetes.csv"
df = pd.read_csv(url)

# X: todas menos a última | y: última
X = df.iloc[:, :-1].to_numpy(float)
y = df.iloc[:, -1].to_numpy(int)

# Instancia com os MESMOS pesos “chutados” do teu código (bias=-130, w1=1, w2=0)
clf = Perceptron(w=[-130, 1, 0])

# net e out
net = clf.net(X)
print("net:\n", net)

out = clf.predict(X)
print("out:\n", out)

# acurácia
acc = (out == y).mean()
print(f"Acurácia com w atual: {acc:.3f}")


net:
 [-45. -41. -14. -33. -42. -38.  -8. -27. -24. -59. -27. -29. -42. -57.
 -86. -31. -35. -18. -17. -47. -29. -23. -50. -59. -37.  -5. -49.  -4.
 -34. -33. -31. -34. -37.  -2. -40. -16. -31. -21. -42. -30. -10. -19.
 -43. -55. -43. -57. -19. -45. -25. -22.  18.   7.  67.  38.  59.  66.
  13.  17.  28.  41.  50.  46.  57.   3.   1.   7.   6.  33.  41.  25.
  30.  16.  32.  40.  26.  58.  22.  33.   1.   4.  49.  64.  51.   9.
  29.   5.  28.  18.  66.  32.  54.  10.  21.  47.  28.  32.  12.   4.
  41.  51.]
out:
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Acurácia com w atual: 1.000


### Perceptron com Hebbian Learning

In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

class Perceptron:
    def __init__(self, taxa=0.1, epocas=1000):
        self.taxa = taxa
        self.epocas = epocas
        self.w = None
        self.historico_acc = []

    @staticmethod
    def _add_bias(X):
        return np.concatenate([np.ones((X.shape[0], 1)), X], axis=1)

    @staticmethod
    def _heaviside(z):
        return (z >= 0).astype(int)

    def fit(self, X, y, plot=False):
        Xb = self._add_bias(X)
        self.w = np.zeros(Xb.shape[1])
        melhor_acc, melhor_w = 0, self.w.copy()

        for _ in range(self.epocas):
            erros = 0
            idx = np.arange(len(Xb))
            np.random.shuffle(idx)
            for i in idx:
                net = np.dot(Xb[i], self.w)
                out = np.heaviside(net, 0)
                erro = y[i] - out
                if erro != 0:
                    self.w += self.taxa * erro * Xb[i]
                    acc = (self.predict(X) == y).mean()
                    self.historico_acc.append(acc)
                    if acc > melhor_acc:
                        melhor_acc, melhor_w = acc, self.w.copy()
                    erros += 1
            if erros == 0:
                print("Convergiu!")
                break

        self.w = melhor_w
        print(f"\nMelhor acurácia: {melhor_acc:.3f}")
        print(f"Pesos finais: {self.w}")
        if plot and self.historico_acc:
            plt.plot(self.historico_acc)
            plt.xlabel("Atualizações")
            plt.ylabel("Acurácia")
            plt.title("Evolução da Acurácia")
            plt.grid(True)
            plt.show()
        return self

    def predict(self, X):
        Xb = self._add_bias(X)
        return self._heaviside(np.dot(Xb, self.w))

    def score(self, X, y):
        return (self.predict(X) == y).mean()


# --- Execução direta ---
url = "https://raw.githubusercontent.com/Zewtta/PEL219_2025_RedesNeuraisArtificiais/e5a63434c13a8ac9a80d4d303b97a8c8de5b01d0/simple_diabetes.csv"
df = pd.read_csv(url)
X = df.iloc[:, :-1].to_numpy(float)
y = df.iloc[:, -1].to_numpy(int)

# Normaliza (opcional, melhora convergência)
X = (X - X.mean(axis=0)) / X.std(axis=0)

modelo = Perceptron(taxa=0.3, epocas=3000).fit(X, y, plot=False)
acc = modelo.score(X, y)
print(f"\nAcurácia final: {acc:.3f}")


Convergiu!

Melhor acurácia: 1.000
Pesos finais: [0.         0.43290902 0.3389458 ]

Acurácia final: 1.000


### Perceptron Hebbian Learning com 80/20

In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

class Perceptron:
    def __init__(self, taxa=0.1, epocas=1000):
        self.taxa = taxa
        self.epocas = epocas
        self.w = None
        self.historico_acc = []

    @staticmethod
    def _add_bias(X):
        return np.concatenate([np.ones((X.shape[0], 1)), X], axis=1)

    @staticmethod
    def _heaviside(z):
        return (z >= 0).astype(int)

    def fit(self, X, y, plot=False):
        Xb = self._add_bias(X)
        self.w = np.zeros(Xb.shape[1])
        melhor_acc, melhor_w = 0, self.w.copy()

        for _ in range(self.epocas):
            erros = 0
            idx = np.arange(len(Xb))
            np.random.shuffle(idx)
            for i in idx:
                net = np.dot(Xb[i], self.w)
                out = np.heaviside(net, 0)
                erro = y[i] - out
                if erro != 0:
                    self.w += self.taxa * erro * Xb[i]
                    acc = (self.predict(X) == y).mean()
                    self.historico_acc.append(acc)
                    if acc > melhor_acc:
                        melhor_acc, melhor_w = acc, self.w.copy()
                    erros += 1
            if erros == 0:
                print("Convergiu!")
                break

        self.w = melhor_w
        print(f"\nMelhor acurácia (treino): {melhor_acc:.3f}")
        print(f"Pesos finais: {self.w}")
        if plot and self.historico_acc:
            plt.plot(self.historico_acc)
            plt.xlabel("Atualizações")
            plt.ylabel("Acurácia")
            plt.title("Evolução da Acurácia (treino)")
            plt.grid(True)
            plt.show()
        return self

    def predict(self, X):
        Xb = self._add_bias(X)
        return self._heaviside(np.dot(Xb, self.w))

    def score(self, X, y):
        return (self.predict(X) == y).mean()


# --- Leitura dos dados
url = "https://raw.githubusercontent.com/Zewtta/PEL219_2025_RedesNeuraisArtificiais/e5a63434c13a8ac9a80d4d303b97a8c8de5b01d0/simple_diabetes.csv"
df = pd.read_csv(url)
X = df.iloc[:, :-1].to_numpy(float)
y = df.iloc[:, -1].to_numpy(int)

# Normalização (melhor convergência)
X = (X - X.mean(axis=0)) / X.std(axis=0)

# --- Divisão treino/teste 80/20
rng = np.random.default_rng(42)
idx = rng.permutation(len(X))
cut = int(0.8 * len(X))
X_tr, X_te = X[idx[:cut]], X[idx[cut:]]
y_tr, y_te = y[idx[:cut]], y[idx[cut:]]

# --- Treinamento
modelo = Perceptron(taxa=0.1, epocas=50).fit(X_tr, y_tr, plot=False)

# --- Avaliação
acc_tr = modelo.score(X_tr, y_tr)
acc_te = modelo.score(X_te, y_te)

print(f"\nAcurácia treino: {acc_tr:.3f}")
print(f"Acurácia teste : {acc_te:.3f}")


Convergiu!

Melhor acurácia (treino): 1.000
Pesos finais: [0.         0.18242078 0.10168374]

Acurácia treino: 1.000
Acurácia teste : 1.000


### Verificação do modelo

In [71]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Carregar base Pima Indians ---
url = "https://raw.githubusercontent.com/Zewtta/PEL219_2025_RedesNeuraisArtificiais/e5a63434c13a8ac9a80d4d303b97a8c8de5b01d0/simple_diabetes.csv"
df = pd.read_csv(url)

X = df.iloc[:, :-1].to_numpy(float)
y = df.iloc[:, -1].to_numpy(int)

# Separar treino/teste (80/20) ---
X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Normalização (só com base no treino) ---
scaler = StandardScaler()
Xtr_n = scaler.fit_transform(X_tr)
Xte_n = scaler.transform(X_te)

# Treinar Perceptron (sklearn) ---
modelo = Perceptron(
    penalty=None,     # sem regularização (como o perceptron “clássico”)
    alpha=0.0001,     # ignorado se penalty=None
    max_iter=100,
    eta0=0.1,         # taxa de aprendizado
    tol=None,         # desliga early stopping
    shuffle=True,
    random_state=42
)

modelo.fit(Xtr_n, y_tr)

# Avaliar ---
ytr_pred = modelo.predict(Xtr_n)
yte_pred = modelo.predict(Xte_n)

acc_tr = accuracy_score(y_tr, ytr_pred)
acc_te = accuracy_score(y_te, yte_pred)

print(f"Acurácia treino: {acc_tr:.3f}")
print(f"Acurácia teste : {acc_te:.3f}")

print("\nRelatório (teste):")
print(classification_report(y_te, yte_pred, digits=3))

print("Matriz de confusão (teste):")
print(confusion_matrix(y_te, yte_pred))


Acurácia treino: 1.000
Acurácia teste : 1.000

Relatório (teste):
              precision    recall  f1-score   support

           0      1.000     1.000     1.000        10
           1      1.000     1.000     1.000        10

    accuracy                          1.000        20
   macro avg      1.000     1.000     1.000        20
weighted avg      1.000     1.000     1.000        20

Matriz de confusão (teste):
[[10  0]
 [ 0 10]]


## Atividade 1 - Treinamento com base completa Diabetes

In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ------------------ Perceptron POO ------------------
class Perceptron:
    def __init__(self, taxa=0.3, epocas=3000):
        self.taxa = taxa
        self.epocas = epocas
        self.w = None
        self.historico_acc = []

    @staticmethod
    def _add_bias(X):
        return np.concatenate([np.ones((X.shape[0], 1)), X], axis=1)

    @staticmethod
    def _heaviside(z):
        return (z >= 0).astype(int)

    def fit(self, X, y, plot=False):
        Xb = self._add_bias(X)
        self.w = np.zeros(Xb.shape[1])
        melhor_acc, melhor_w = 0.0, self.w.copy()

        for _ in range(self.epocas):
            erros = 0
            idx = np.arange(len(Xb))
            np.random.shuffle(idx)
            for i in idx:
                net = np.dot(Xb[i], self.w)
                out = np.heaviside(net, 0)
                e = y[i] - out
                if e != 0:
                    self.w += self.taxa * e * Xb[i]
                    acc = (self.predict(X) == y).mean()
                    self.historico_acc.append(acc)
                    if acc > melhor_acc:
                        melhor_acc, melhor_w = acc, self.w.copy()
                    erros += 1
            if erros == 0:
                print("Convergiu!")
                break

        self.w = melhor_w
        print(f"\nMelhor acurácia (treino): {melhor_acc:.3f}")
        if plot and self.historico_acc:
            plt.figure()
            plt.plot(self.historico_acc)
            plt.xlabel("Atualizações")
            plt.ylabel("Acurácia (treino)")
            plt.title("Evolução da Acurácia")
            plt.grid(True)
            plt.show()
        return self

    def predict(self, X):
        Xb = self._add_bias(X)
        return self._heaviside(np.dot(Xb, self.w))

    def score(self, X, y):
        return (self.predict(X) == y).mean()


# ------------------ Carregar base completa ------------------
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
cols = ["preg","plas","pres","skin","insu","mass","pedi","age","class"]
df = pd.read_csv(url, header=None, names=cols)

X_all = df.iloc[:, :-1].to_numpy(float)  # 8 features
y_all = df.iloc[:, -1].to_numpy(int)     # rótulo 0/1

# ------------------ Split 80/20 (sem sklearn) ------------------
rng = np.random.default_rng(42)
idx = rng.permutation(len(X_all))
cut = int(0.8 * len(X_all))
tr, te = idx[:cut], idx[cut:]

X_tr, y_tr = X_all[tr], y_all[tr]
X_te, y_te = X_all[te], y_all[te]

# ------------------ Normalização (usa APENAS treino) ------------------
mu = X_tr.mean(axis=0, keepdims=True)
sd = X_tr.std(axis=0, keepdims=True) + 1e-12
Xtr_n = (X_tr - mu) / sd
Xte_n = (X_te - mu) / sd

# ------------------ Treinar e avaliar ------------------
modelo = Perceptron(taxa=0.1, epocas=100).fit(Xtr_n, y_tr, plot=False)

acc_tr = modelo.score(Xtr_n, y_tr)
acc_te = modelo.score(Xte_n, y_te)

print(f"\nAcurácia treino: {acc_tr:.3f}")
print(f"Acurácia teste : {acc_te:.3f}")
print(f"Pesos finais (bias + 8):\n{modelo.w}")



Melhor acurácia (treino): 0.777

Acurácia treino: 0.777
Acurácia teste : 0.805
Pesos finais (bias + 8):
[-0.4         0.10221959  0.6029698  -0.0620309   0.06875877  0.09180629
  0.37699755  0.23338484  0.0721811 ]


### Verificação do modelo

In [87]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Carregar base Pima Indians ---
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
cols = ["preg","plas","pres","skin","insu","mass","pedi","age","class"]
df = pd.read_csv(url, header=None, names=cols)

X = df.iloc[:, :-1].to_numpy(float)
y = df.iloc[:, -1].to_numpy(int)

# Separar treino/teste (80/20) ---
X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Normalização (só com base no treino) ---
scaler = StandardScaler()
Xtr_n = scaler.fit_transform(X_tr)
Xte_n = scaler.transform(X_te)

# Treinar Perceptron (sklearn) ---
modelo = Perceptron(
    penalty=None,     # sem regularização (como o perceptron “clássico”)
    alpha=0.0001,     # ignorado se penalty=None
    max_iter=50000,
    eta0=0.1,         # taxa de aprendizado
    tol=True,         # desliga early stopping
    shuffle=True,
    random_state=42
)

modelo.fit(Xtr_n, y_tr)

# Avaliar ---
ytr_pred = modelo.predict(Xtr_n)
yte_pred = modelo.predict(Xte_n)

acc_tr = accuracy_score(y_tr, ytr_pred)
acc_te = accuracy_score(y_te, yte_pred)

print(f"Acurácia treino: {acc_tr:.3f}")
print(f"Acurácia teste : {acc_te:.3f}")

print("\nRelatório (teste):")
print(classification_report(y_te, yte_pred, digits=3))

print("Matriz de confusão (teste):")
print(confusion_matrix(y_te, yte_pred))


Acurácia treino: 0.738
Acurácia teste : 0.727

Relatório (teste):
              precision    recall  f1-score   support

           0      0.784     0.800     0.792       100
           1      0.615     0.593     0.604        54

    accuracy                          0.727       154
   macro avg      0.700     0.696     0.698       154
weighted avg      0.725     0.727     0.726       154

Matriz de confusão (teste):
[[80 20]
 [22 32]]
